# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  4.81it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.64it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.37it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.23it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.38it/s]



...

 70%|██████▉   | 16/23 [00:03<00:01,  5.30it/s]

 78%|███████▊  | 18/23 [00:04<00:00,  5.17it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  5.30it/s]

 96%|█████████▌| 22/23 [00:04<00:00,  5.35it/s]

100%|██████████| 23/23 [00:05<00:00,  4.46it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Richard. I am a 40-year-old web developer who loves to learn and explore new things. I have a degree in Computer Science and have been working in the industry for over 15 years. I'm a bit of a nerd, but I like to think that's a good thing!

In my free time, I enjoy playing video games, watching movies, and reading science fiction novels. I'm also a bit of a coffee snob and love trying out new coffee shops in my area.

I'm excited to start this blog and share my thoughts and experiences with the world. I'm not sure what topics I'll cover, but
Prompt: The president of the United States is
Generated text:  not a monarch, but a public servant with a specific set of duties and responsibilities. One of those duties is to nominate a new justice to the Supreme Court when a vacancy arises, a power granted to the president by Article II, Section 2 of the Constitution. The Senate then advises and consents on the nomination, which is typically a lengthy p

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Nick

y

 Wat

erson

...

,

 we

're

 seeing

 a

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Sophia and I am a 22-year-old college student. I am a senior in a small liberal arts college in the Midwest, and I am planning to graduate in May with a degree in English literature. I have a passion for writing, reading, and learning, and I am excited to share my experiences and perspectives with you.
Throughout my college career, I have been involved in various extracurricular activities that have allowed me to grow both personally and professionally. As an English major, I have had the opportunity to take a wide range of courses that have helped me develop my critical thinking, writing, and communication skills. I have also

Prompt: The capital of France is
Generated text:  often called the "City of Light" and the most visited city in the world. Paris is renowned for its stunning architecture, artistic heritage, and world-class cuisine. The Eiffel Tower, the Louvre, the Seine River, and Notre-Dame Cathedral are just a few of the many famou

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Maya

 and

 I

 am

...

 including

 books

,

 articles

,

In [6]:
llm.shutdown()